In [1]:
import findspark
findspark.init()
import pyspark
import os

sc = pyspark.SparkContext()
spark = pyspark.SQLContext(sc)

working_dir = os.getcwd()

In [2]:
# Load parquet file into dataframe
df = spark.read.parquet("file://" + working_dir + "/testDataset/part-00000-711fabb0-5efc-4d83-afad-0e03a3156794.snappy.parquet")

# Exploration and data cleansing

#### Show total count of lines

In [3]:
# Count lines
df.count()

600000

#### Show the aspect of fiew records

In [4]:
# Show first 5 lines
df.show(5)

+--------------------+--------------------+--------------------+---------+--------+---------+
|            deviceId|             eventId|           eventTime|      lat|     lon|eventType|
+--------------------+--------------------+--------------------+---------+--------+---------+
|8f88ac99-8aa5-88a...|000c5bd2-d2d5-4cf...|2017-05-24 12:34:...|47.409271|8.547062| location|
|8f88ac99-8aa5-88a...|001b3982-ec22-4bb...|2017-05-24 16:21:...|47.417977|8.554384| location|
|8f88ac99-8aa5-88a...|00287463-b763-40b...|2017-05-24 16:02:...|47.416406| 8.55298| location|
|8f88ac99-8aa5-88a...|002946bb-19b8-40a...|2017-05-24 15:03:...|47.417743|8.554184| location|
|8f88ac99-8aa5-88a...|002947fd-bb93-434...|2017-05-24 12:56:...|47.406545|8.547231| location|
+--------------------+--------------------+--------------------+---------+--------+---------+
only showing top 5 rows



#### Show all types of eventType

In [5]:
df.createOrReplaceTempView("rawdata")
spark.sql("SELECT DISTINCT eventType FROM rawdata").show()

+-------------+
|    eventType|
+-------------+
|status-update|
|     trip-end|
|     location|
|   trip-start|
|   mil-update|
+-------------+



#### Remove rows with null or NaN values

In [6]:
# Remove rows with any null or NaN value
df.dropna
# Count lines
df.count()

600000

In [7]:
# Update created temp view for raw data
df.createOrReplaceTempView("rawdata")

### Remove unnecessary event types
Since we are going to predict estimation based on the start position, we won't need the rest of events (location, etc). Also, we remove rows with gps non valid values (0.0).

In [8]:
# 1. Create a view with only the values we need for processing
df = spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM rawdata \
WHERE (eventType='trip-start' OR eventType='trip-end') \
AND (lat != 0.0 and lon != 0.0) ORDER BY eventTime")
df.createOrReplaceTempView("cleands")
df.count()

13444

## Explore data transformation for a single device
The purpose is to reduce the problem so that we are sure that the transformations that we apply to the data produce the desired results. 
The aim of these transformations is to clean the dataset and take only the correctly paired start-trip / end-trip data.

In [9]:
# Show an example of data for a certain device
singleDevice = spark.sql("SELECT * FROM cleands WHERE deviceId='D08699AE-BDAC-4AB4-2F15-177C74993133'")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100, False)

+------------------------------------+---------------------+---------+----------+----------+
|deviceId                            |eventTime            |lat      |lon       |eventType |
+------------------------------------+---------------------+---------+----------+----------+
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-25 23:46:05.0|32.988837|-97.263735|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-25 23:46:09.0|32.988837|-97.263735|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 00:35:30.0|32.98925 |-97.263821|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 00:35:39.0|32.989253|-97.263822|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 02:08:40.0|32.841525|-97.068658|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 02:08:46.0|32.841525|-97.068658|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 13:57:30.0|32.926142|-97.087224|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 19:17:41.0|32.98928 |

### 1. Extract previous and next event for every row
To do this, we use LAG and LEAD window functions to shift the data. The result will add two new columns *previousEvent* and *nextEvent* to the dataset

In [10]:
# Shift values of eventType to add previous and next event
singleDevice = spark.sql("SELECT *, LAG(eventType, 1) OVER (ORDER BY eventTime) AS previousEvent, \
LEAD(eventType, 1) OVER (ORDER BY eventTime) AS nextEvent FROM singledevice")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100)

+--------------------+--------------------+---------+----------+----------+-------------+----------+
|            deviceId|           eventTime|      lat|       lon| eventType|previousEvent| nextEvent|
+--------------------+--------------------+---------+----------+----------+-------------+----------+
|D08699AE-BDAC-4AB...|2017-05-25 23:46:...|32.988837|-97.263735|  trip-end|         null|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-25 23:46:...|32.988837|-97.263735|  trip-end|     trip-end|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-26 00:35:...| 32.98925|-97.263821|  trip-end|     trip-end|trip-start|
|D08699AE-BDAC-4AB...|2017-05-26 00:35:...|32.989253|-97.263822|trip-start|     trip-end|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-26 02:08:...|32.841525|-97.068658|  trip-end|   trip-start|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-26 02:08:...|32.841525|-97.068658|  trip-end|     trip-end|trip-start|
|D08699AE-BDAC-4AB...|2017-05-26 13:57:...|32.926142|-97.087224|trip-start|     trip-end|tr

### 2. Remove trip-start and trip-end non valid rows
The only valid data is that which defines a valid trip, that is, a trip-start followed by a trip-end event. Those trip-start with no trip-end after it, or those trip-end without a preceding trip-start are removed from the dataset.

In [11]:
# Remove non valid trip-start and trip-end (those without its corresponding pair)
singleDevice = spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM singledevice \
WHERE NOT ((eventType='trip-end' AND previousEvent='trip-end') \
OR (eventType='trip-start' AND nextEvent='trip-start'))")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(20, False)


+------------------------------------+---------------------+---------+----------+----------+
|deviceId                            |eventTime            |lat      |lon       |eventType |
+------------------------------------+---------------------+---------+----------+----------+
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 00:35:39.0|32.989253|-97.263822|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 02:08:40.0|32.841525|-97.068658|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 19:17:41.0|32.98928 |-97.264482|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 19:19:30.0|32.988822|-97.263723|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-01 00:11:22.0|32.989214|-97.26381 |trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-01 00:38:47.0|32.841445|-97.06854 |trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-01 13:44:54.0|32.789698|-97.133493|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-02 01:17:43.0|32.84152 |

### 3. Put all the data of every complete trip in the same row
We move the position and time data of every complete trip in a single row. This way we will be able to process each row as a trip, and we'll no longer need deviceId or eventType

In [12]:
# Move start and end values of a trip to a single row. We won't need deviceId any more
singleDevice = spark.sql("SELECT eventTime AS eventTimeStart, \
LEAD(eventTime, 1) OVER (ORDER BY eventTime) AS eventTimeEnd, lat AS latStart, lon AS lonStart, \
LEAD(lat, 1) OVER (ORDER BY eventTime) AS latEnd, \
LEAD(lon, 1) OVER (ORDER BY eventTime) AS lonEnd, \
eventType FROM singledevice")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice = spark.sql("SELECT eventTimeStart, eventTimeEnd, latStart, lonStart, latEnd, lonEnd \
FROM singledevice WHERE eventType='trip-start'")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100, False)

+---------------------+---------------------+---------+----------+---------+----------+
|eventTimeStart       |eventTimeEnd         |latStart |lonStart  |latEnd   |lonEnd    |
+---------------------+---------------------+---------+----------+---------+----------+
|2017-05-26 00:35:39.0|2017-05-26 02:08:40.0|32.989253|-97.263822|32.841525|-97.068658|
|2017-05-26 19:17:41.0|2017-05-26 19:19:30.0|32.98928 |-97.264482|32.988822|-97.263723|
|2017-06-01 00:11:22.0|2017-06-01 00:38:47.0|32.989214|-97.26381 |32.841445|-97.06854 |
|2017-06-01 13:44:54.0|2017-06-02 01:17:43.0|32.789698|-97.133493|32.84152 |-97.068662|
|2017-06-02 14:07:44.0|2017-06-02 14:32:29.0|32.841552|-97.068303|32.988817|-97.263714|
|2017-06-08 02:19:02.0|2017-06-08 13:53:49.0|32.822621|-97.058403|32.847142|-97.076774|
|2017-06-09 00:04:53.0|2017-06-09 00:05:51.0|32.841374|-97.067877|32.841491|-97.068668|
|2017-06-10 00:58:05.0|2017-06-10 01:23:09.0|32.989001|-97.263582|32.841531|-97.068663|
+---------------------+---------

## Apply the transformations to the whole dataset

### 1. Extract previous and next event for every row
To do this, we use LAG and LEAD window functions to shift the data. The result will add two new columns *previousEvent* and *nextEvent* to the dataset

In [13]:
# Shift values of eventType to add previous and next event
allDevices = spark.sql("SELECT *, LAG(eventType, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS previousEvent, \
LEAD(eventType, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS nextEvent FROM cleands")
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20)

+--------------------+--------------------+----------+---------+----------+-------------+---------+
|            deviceId|           eventTime|       lat|      lon| eventType|previousEvent|nextEvent|
+--------------------+--------------------+----------+---------+----------+-------------+---------+
|11111111-1b2a-4bc...|2017-03-22 16:50:...|38.7735368|-9.168737|trip-start|         null| trip-end|
|11111111-1b2a-4bc...|2017-03-22 17:14:...|  38.76522|-9.098054|  trip-end|   trip-start| trip-end|
|11111111-1b2a-4bc...|2017-03-22 17:14:...|  38.76522|-9.098054|  trip-end|     trip-end|     null|
|11111111-2605-430...|2017-03-22 17:14:...|  38.76522|-9.098054|  trip-end|         null| trip-end|
|11111111-2605-430...|2017-03-22 17:14:...|  38.76522|-9.098054|  trip-end|     trip-end|     null|
|11111111-3306-4d3...|2017-03-22 16:50:...|38.7735368|-9.168737|trip-start|         null| trip-end|
|11111111-3306-4d3...|2017-03-22 17:14:...|  38.76522|-9.098054|  trip-end|   trip-start| trip-end|


### 2. Remove trip-start and trip-end non valid rows
The only valid data is that which defines a valid trip, that is, a trip-start followed by a trip-end event. Those trip-start with no trip-end after it, or those trip-end without a preceeding trip-start are removed from the dataset.

In [14]:
# Remove non valid trip-start and trip-end (those without its corresponding pair)
allDevices = spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM alldevices \
WHERE NOT ((eventType='trip-end' AND previousEvent='trip-end') \
OR (eventType='trip-start' AND nextEvent='trip-start'))")
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20, False)

+------------------------------------+---------------------+----------+---------+----------+
|deviceId                            |eventTime            |lat       |lon      |eventType |
+------------------------------------+---------------------+----------+---------+----------+
|11111111-1b2a-4bc0-9818-2b62e59a1a4a|2017-03-22 16:50:25.0|38.7735368|-9.168737|trip-start|
|11111111-1b2a-4bc0-9818-2b62e59a1a4a|2017-03-22 17:14:26.0|38.76522  |-9.098054|trip-end  |
|11111111-3306-4d31-b992-cd8b79d45e58|2017-03-22 16:50:25.0|38.7735368|-9.168737|trip-start|
|11111111-3306-4d31-b992-cd8b79d45e58|2017-03-22 17:14:26.0|38.76522  |-9.098054|trip-end  |
|11111111-74d8-4eb3-ae71-693eb56cc990|2017-03-22 16:50:25.0|38.7735368|-9.168737|trip-start|
|11111111-74d8-4eb3-ae71-693eb56cc990|2017-03-22 17:14:26.0|38.76522  |-9.098054|trip-end  |
|11111111-a924-49c8-84b4-60b0c2e7b5d3|2017-03-22 16:50:25.0|38.7735368|-9.168737|trip-start|
|11111111-a924-49c8-84b4-60b0c2e7b5d3|2017-03-22 17:14:26.0|38.76522  

In [15]:
allDevices.count()

6308

### 3. Put all the data of every complete trip in the same row
We move the position and time data of every complete trip in a single row. This way we will be able to process each row as a trip, and we'll no longer need deviceId or eventType

In [16]:
# Move start and end values of a trip to a single row. We won't need deviceId any more
allDevices = spark.sql("SELECT eventTime AS eventTimeStart, \
LEAD(eventTime, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS eventTimeEnd, lat AS latStart, lon AS lonStart, \
LEAD(lat, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS latEnd, \
LEAD(lon, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS lonEnd, \
eventType FROM alldevices")
allDevices.createOrReplaceTempView("alldevices")
allDevices = spark.sql("SELECT eventTimeStart, eventTimeEnd, latStart, lonStart, latEnd, lonEnd \
FROM alldevices WHERE eventType='trip-start'")
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20, False)

+---------------------+---------------------+----------+---------+--------+---------+
|eventTimeStart       |eventTimeEnd         |latStart  |lonStart |latEnd  |lonEnd   |
+---------------------+---------------------+----------+---------+--------+---------+
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.7735368|-9.168737|38.76522|-9.098054|
|2017-03-22 16:50:25.0|2017-03-22 17:14:26.0|38.773536

In [17]:
allDevices.count()

3154

### Remove duplicates
After examining the resulting data, we observe that there many registers with exactly the same data for the time start and de starting and end position, so we remove the redundant data to avoid bias on the predicting model

In [18]:
allDevices = allDevices.dropDuplicates(['eventTimeStart', 'latStart', 'lonStart', 'latEnd', 'lonEnd'])
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20, False)

+---------------------+---------------------+---------+--------+---------+----------+
|eventTimeStart       |eventTimeEnd         |latStart |lonStart|latEnd   |lonEnd    |
+---------------------+---------------------+---------+--------+---------+----------+
|2017-05-31 14:00:36.0|2017-05-31 14:07:38.0|33.049029|-96.7165|33.040753|-96.736455|
|2017-05-24 12:30:37.0|2017-05-24 12:30:37.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 13:00:22.0|2017-05-24 13:00:23.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 15:58:39.0|2017-05-24 15:58:40.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 16:33:43.0|2017-05-24 16:33:43.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 12:27:10.0|2017-05-24 12:27:10.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 15:36:09.0|2017-05-24 15:36:09.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 15:42:32.0|2017-05-24 15:42:33.0|47.409291|8.546942|47.423743|8.555213  |
|2017-05-24 17:42:37.0|2017-05-24 17:42:37.0|47.409291

In [19]:
allDevices.count()

1537

# Data visualization

In [20]:
import pandas as pd
dfProcessed = allDevices.toPandas()

### Data check after transformations
We make an overview of the description of the data before and after the processing to check that it makes sense.

First we describe the data after processing. We observe allmost all the position data for start and end is identical, so just looking at the data we could tell that our car is always doing the same trip.

In [21]:
dfProcessed.describe()

,latStart,lonStart,latEnd,lonEnd
count,1537.000000,1537.000000,1537.000000,1537.000000
mean,47.196908,7.022652,47.210831,7.031469
std,1.729804,12.564383,1.734106,12.560133
min,32.789698,-97.264482,32.841445,-97.302954
25%,47.409291,8.546942,47.423743,8.555213
50%,47.409291,8.546942,47.423743,8.555213
75%,47.409291,8.546942,47.423743,8.555213
max,47.409291,8.546942,47.423743,8.555213


Then we also describe the data before doing any processing. Starting and end positions are mixed, so we can make much conclusions, but we can observe that the range of the data make sense.

In [22]:
dfRaw = df.toPandas()
dfRaw.describe()

,lat,lon
count,13444.000000,13444.000000
mean,43.070245,-0.743264
std,4.376916,11.331070
min,32.789698,-97.302954
25%,38.765220,-9.098054
50%,47.409291,8.546942
75%,47.409291,8.546942
max,47.423743,8.555213


In [23]:
dfProcessed = dfProcessed.sort_values("eventTimeStart")
dfProcessed

,eventTimeStart,eventTimeEnd,latStart,lonStart,latEnd,lonEnd
162,2017-03-22 16:50:25,2017-03-22 17:14:26,38.773537,-9.168737,38.765220,-9.098054
136,2017-05-24 12:20:56,2017-05-24 12:21:02,47.409291,8.546942,47.423743,8.555213
589,2017-05-24 12:21:08,2017-05-24 12:21:13,47.409291,8.546942,47.423743,8.555213
1505,2017-05-24 12:21:17,2017-05-24 12:21:21,47.409291,8.546942,47.423743,8.555213
524,2017-05-24 12:21:24,2017-05-24 12:21:31,47.409291,8.546942,47.423743,8.555213
609,2017-05-24 12:21:32,2017-05-24 12:21:34,47.409291,8.546942,47.423743,8.555213
1116,2017-05-24 12:21:37,2017-05-24 12:21:38,47.409291,8.546942,47.423743,8.555213
1355,2017-05-24 12:22:08,2017-05-24 12:22:09,47.409291,8.546942,47.423743,8.555213
1258,2017-05-24 12:22:23,2017-05-24 12:22:26,47.409291,8.546942,47.423743,8.555213
373,2017-05-24 12:22:27,2017-05-24 12:22:32,47.409291,8.546942,47.423743,8.555213


## Map visualization
When handling geospatial data, is a good practise to visualize it in a map to find insights or avoid erroneous assumptions on the data

In [24]:
# A google maps Javascript API KEY is needed to see the maps in the notebook
import gmaps
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [25]:
startLocs = dfProcessed[['latStart','lonStart']]
endLocs = dfProcessed[['latEnd', 'lonEnd']]
center_coords = (32.920782, -96.950904)

### Starting positions visualization
A heatmap is displayed to observe the density of the trip-start positions in a map. We can see that there are three main areas around Dallas area were the cars start their trips.

In [26]:
fig = gmaps.figure(center=center_coords, zoom_level=10)
heatmap = gmaps.heatmap_layer(startLocs)
heatmap.max_intensity = 2
heatmap.point_radius = 20
fig.add_layer(heatmap)
fig

### Ending positions visualization
As with starting positions, a heatmap is displayed to observe the density of the trip-end positions in a map. Again, there three main areas were the cars end their trips, showing that they likely do a go and return trip on the same areas.

In [27]:
fig = gmaps.figure(center=center_coords, zoom_level=10)
heatmap = gmaps.heatmap_layer(endLocs)
heatmap.max_intensity = 2
heatmap.point_radius = 20
fig.add_layer(heatmap)
fig

Below, the positions markers combined are shown. trip-start are shown in green and trip-end in red

In [39]:
center_coords = (32.874843, -96.947014)
figMarkers = gmaps.figure(center=center_coords, zoom_level=10)
trip_start_layer = gmaps.symbol_layer(
startLocs, fill_color="rgba(0, 150, 0, 0.4)",
stroke_color="rgba(0, 150, 0, 0.4)", scale=4
)
trip_end_layer = gmaps.symbol_layer(
endLocs, fill_color="rgba(200, 0, 0, 0.4)",
stroke_color="rgba(200, 0, 0, 0.4)", scale=4
)
#trip_start_layer = gmaps.symbol_layer(startLocs, fill_color="green", stroke_color="green", scale=3)
#trip_end_layer = gmaps.symbol_layer(endLocs, , stroke_color="red", scale=3, fill_color=None, fill_opacity=0.6)
figMarkers.add_layer(trip_start_layer)
figMarkers.add_layer(trip_end_layer)
figMarkers

After cleansing and processing we save the processed dataset to a csv file for its later use to build the prediction model.

In [29]:
dfProcessed.to_csv('processed-dataset.csv')